In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import yaml
import logging
import pandas as pd
import numpy as np
import scipy
import seaborn as sns
import matplotlib.pyplot as plt

sys.path.append('../')

from abacus.auto_ab.abtest import ABTest
from abacus.auto_ab.params import ABTestParams
from abacus.auto_ab.params import DataParams, HypothesisParams
from abacus.splitter.split_builder import SplitBuilder
from abacus.splitter.params import SplitBuilderParams
from abacus.mde_researcher.params import MdeParams
from abacus.mde_researcher.mde_research_builder import MdeResearchBuilder
from abacus.mde_researcher.multiple_split_builder import MultipleSplitBuilder

logging.basicConfig(level = logging.INFO)


%load_ext autoreload
%autoreload 2

# Данные, с которыми будем работать

In [ ]:
pd.read_csv('./data/ab_data.csv').head()

# Кейс №1. Постанализ непрерывной метрики.

In [ ]:
df = pd.read_csv('./data/ab_data.csv')

In [ ]:
data_params = DataParams(
    id_col='id', 
    group_col='groups', 
    target='height_now', 
    is_grouped=True
)

hypothesis_params = HypothesisParams(
    alpha=0.05, 
    beta=0.2, 
    alternative='greater', 
    metric_type='continuous', 
    metric_name='mean', 
    metric=np.mean, 
    n_boot_samples=100, 
)

ab_params = ABTestParams(data_params,
                         hypothesis_params)

In [ ]:
ab_test = ABTest(df, ab_params)

In [ ]:
print('Нет эффекта:\n')

print(f"bootstrap_test: {ab_test.test_boot_confint()}")
print(f"mannwhitney_test: {ab_test.test_mannwhitney()}")
print(f"welch_test: {ab_test.test_welch()}")

Увеличиваем количество прокрасов в тестовой группе

In [ ]:
df[ab_test.params.data_params.target].where(df[ab_test.params.data_params.group_col] == 'A',
                                            df[ab_test.params.data_params.target] * 1.001, 
                                            axis=0,
                                            inplace=True)

ab_test = ABTest(df, ab_params)

In [ ]:
print('Есть эффект:')
print(f"bootstrap_test: {ab_test.test_boot_confint()}")
print(f"mannwhitney_test: {ab_test.test_mannwhitney()}")
print(f"welch_test: {ab_test.test_welch()}")

In [ ]:
ab_test.plot()

## Кейс №1.1. Постанализ непрерывной метрики с бакетированием.

In [ ]:
df = pd.read_csv('./data/ab_data.csv')
df['height_now'] = np.random.lognormal(1, 0.5, df.shape[0])

In [ ]:
data_params = DataParams(
    id_col='id', 
    group_col='groups', 
    target='height_now', 
    is_grouped=True
)

hypothesis_params = HypothesisParams(
    alpha=0.05, 
    beta=0.2, 
    alternative='two-sided', 
    metric_type='continuous', 
    metric_name='mean', 
    metric=np.mean, 
    n_boot_samples=100, 
    n_buckets=500
)

ab_params = ABTestParams(data_params, hypothesis_params)

In [ ]:
ab_test = ABTest(df, ab_params)
print(ab_test.test_welch())

In [ ]:
ab_test.plot()

In [ ]:
print(len(ab_test.params.data_params.control))

In [ ]:
ab_test_11 = ab_test.bucketing()
print(len(ab_test_11.params.data_params.control))

In [ ]:
print(ab_test_11.test_welch())

In [ ]:
ab_test_11.plot()

In [ ]:
scipy.stats.shapiro(ab_test_11.params.data_params.control)

# Кейс №2. Постанализ с увеличением чувствительности непрерывной метрики. CUPED.

In [ ]:
df = pd.read_csv('./data/ab_data.csv')

In [ ]:
data_params = DataParams(
    id_col='id', 
    group_col='groups', 
    strata_col='country', 
    target='height_now', 
    covariate='height_prev', 
    is_grouped=False
)

hypothesis_params = HypothesisParams(
    alpha=0.05, 
    beta=0.2, 
    alternative='two-sided', 
    metric_type='continuous', 
    metric_name='mean', 
    metric=np.mean, 
    n_boot_samples=100, 
    n_buckets=50
)

ab_params = ABTestParams(data_params, hypothesis_params)
ab_test = ABTest(df, ab_params)

Увеличиваем количество прокрасов в тестовой группе

In [ ]:
df[ab_test.params.data_params.target].where(df[ab_test.params.data_params.group_col] == 'A',
                                            df[ab_test.params.data_params.target] * 1.0001, 
                                            axis=0,
                                            inplace=True)
df['id'] = np.random.randint(0, 150_000, df.shape[0])
df = df.drop_duplicates(['id'])

print('Без понижения дисперсии:')
ab_test = ABTest(df, ab_params)
print(f"bootstrap_test: {ab_test.test_boot_confint()}")
print(f"mannwhitney_test: {ab_test.test_mannwhitney()}")
print(f"ttest_test: {ab_test.test_welch()}")

print('\nС понижением дисперсии:')
ab_test1 = ab_test.cuped()
print(f"bootstrap_test: {ab_test1.test_boot_confint()}")
print(f"mannwhitney_test: {ab_test1.test_mannwhitney()}")
print(f"ttest_test: {ab_test1.test_welch()}")

# Кейс №3. Постанализ для бинарной метрики.

In [ ]:
df = pd.read_csv('./data/ab_data.csv')
df.head()

In [ ]:
data_params = DataParams(
    id_col='id', 
    group_col='groups', 
    control_name='A',
    treatment_name='B',
    target_flg='conversion',  
    is_grouped=True
)

hypothesis_params = HypothesisParams(
    alpha=0.05, 
    beta=0.2, 
    alternative='two-sided', 
    metric_type='binary', 
    metric_name='mean', 
)

ab_params = ABTestParams(data_params, hypothesis_params)

In [ ]:
ab_test = ABTest(df, ab_params)

In [ ]:
print('Нет эффекта:')
print(f"z_proportions_test: {ab_test.test_z_proportions()}")

Увеличиваем количество прокрасов в тестовой группе

In [ ]:
df['conversion'] = np.where(df.noise_now < 0, 0, 1)
df[ab_test.params.data_params.target_flg].where(df[ab_test.params.data_params.group_col] == 'A',
                                            np.random.binomial(n=1, p=0.72, size=df.shape[0]),
                                            axis=0,
                                            inplace=True)

ab_test = ABTest(df, ab_params)

print('Есть эффект:')
print(f"z_proportions_test: {ab_test.test_z_proportions()}")

In [ ]:
ab_test.plot()

# Кейс №4 — исключен

# Кейс №5. CUPED.
## Кейс №5.1. Различий между групп нет.

In [ ]:
df = pd.read_csv('./data/ab_data.csv')

In [ ]:
data_params = DataParams(
    id_col='id', 
    target='height_now', 
    covariate='height_prev', 
    is_grouped=True
)

hypothesis_params = HypothesisParams(
    alpha=0.05, 
    beta=0.2, 
    alternative='two-sided',  
    metric_type='continuous', 
    metric_name='mean',
)

ab_params = ABTestParams(data_params, hypothesis_params)

In [ ]:
ab_test = ABTest(df, ab_params)

In [ ]:
print('Control mean = {:.3f}'.format(np.mean(ab_test.params.data_params.control)))
print('Treatment mean = {:.3f}'.format(np.mean(ab_test.params.data_params.treatment)))
print('Control var = {:.3f}'.format(np.var(ab_test.params.data_params.control)))
print('Treatment var = {:.3f}'.format(np.var(ab_test.params.data_params.treatment)))

In [ ]:
print('Различий нет:')
print(f"bootstrap_test: {ab_test.test_boot_confint()}")
print(f"mannwhitney_test: {ab_test.test_mannwhitney()}")
print(f"ttest_test: {ab_test.test_welch()}")

Применяем CUPED:

In [ ]:
ab_test1 = ab_test.cuped()

In [ ]:
print('Control mean = {:.3f}'.format(np.mean(ab_test1.params.data_params.control)))
print('Treatment mean = {:.3f}'.format(np.mean(ab_test1.params.data_params.treatment)))
print('Control var = {:.3f}'.format(np.var(ab_test1.params.data_params.control)))
print('Treatment var = {:.3f}'.format(np.var(ab_test1.params.data_params.treatment)))

In [ ]:
print('Если различий нет, то CUPED и не поможет их найти:')
print(f"bootstrap_test: {ab_test1.test_boot_confint()}")
print(f"mannwhitney_test: {ab_test1.test_mannwhitney()}")
print(f"ttest_test: {ab_test1.test_welch()}")

In [ ]:
ab_test.plot()

In [ ]:
ab_test1.plot()

## Кейс №5.2. Различия между группами есть.

### Увеличиваем значения одной из групп.

In [ ]:
df = pd.read_csv('./data/ab_data.csv')

incs = np.random.uniform(0.0, 0.009, df.shape[0])
df['height_now'].where(df[ab_test.params.data_params.group_col]=='A',
                                    df['height_now'] + incs, 
                                    axis=0,
                                    inplace=True)

In [ ]:
data_params = DataParams(
    id_col='id', 
    group_col='groups',  
    target='height_now', 
    covariate='height_prev', 
    is_grouped=True
)

hypothesis_params = HypothesisParams(
    alpha=0.05, 
    beta=0.2, 
    alternative='greater', 
    metric_type='continuous', 
    metric_name='mean', 
)

ab_params = ABTestParams(data_params, hypothesis_params)

In [ ]:
incs = np.sort(np.random.uniform(0.0, 2, df.shape[0]))
df[ab_test.params.data_params.target].where(df[ab_test.params.data_params.group_col]=='A',
                                    df[ab_test.params.data_params.target] + incs, 
                                    axis=0,
                                    inplace=True)

In [ ]:
ab_test = ABTest(df, ab_params)

In [ ]:
print('Control mean = {:.3f}'.format(np.mean(ab_test.params.data_params.control)))
print('Treatment mean = {:.3f}'.format(np.mean(ab_test.params.data_params.treatment)))
print('Control var = {:.3f}'.format(np.var(ab_test.params.data_params.control)))
print('Treatment var = {:.3f}'.format(np.var(ab_test.params.data_params.treatment)))

In [ ]:
print('Различий нет:\n')

print(f"bootstrap_test: {ab_test.test_boot_confint()}")
print(f"mannwhitney_test: {ab_test.test_mannwhitney()}")
print(f"ttest_test: {ab_test.test_welch()}")

Применяем CUPED:

In [ ]:
ab_test2 = ab_test.cuped()

In [ ]:
print('Control mean = {:.3f}'.format(np.mean(ab_test2.params.data_params.control)))
print('Treatment mean = {:.3f}'.format(np.mean(ab_test2.params.data_params.treatment)))
print('Control var = {:.3f}'.format(np.var(ab_test2.params.data_params.control)))
print('Treatment var = {:.3f}'.format(np.var(ab_test2.params.data_params.treatment)))

In [ ]:
print('Если различия есть, то CUPED может помочь их найти:\n')

print(f"bootstrap_test: {ab_test2.test_boot_confint()}")
print(f"mannwhitney_test: {ab_test2.test_mannwhitney()}")
print(f"ttest_test: {ab_test2.test_welch()}")

In [ ]:
ab_test.plot()

In [ ]:
ab_test2.plot()

# Кейс №6. Всё вместе: chaining rule & reporting.

In [ ]:
df = pd.read_csv('./data/ab_data.csv')
df.head()

In [ ]:
data_params = DataParams(
    id_col='id', 
    group_col='groups', 
    target='height_now', 
    covariate='height_prev', 
    is_grouped=False
)

hypothesis_params = HypothesisParams(
    alpha=0.05, 
    beta=0.2, 
    alternative='two-sided', 
    metric_type='continuous', 
    metric_name='mean', 
    n_boot_samples=100, 
    n_buckets=2000
)

ab_params = ABTestParams(data_params, hypothesis_params)

In [ ]:
df = pd.read_csv('./data/ab_data.csv')
df['height_now'].where(df['groups'] == 'A',
                       df['height_now'] * 1.003,
                       axis=0,
                       inplace=True)
df['id'] = np.random.randint(0, 150_000, df.shape[0])
df = df.drop_duplicates(['id'])

In [ ]:
ab_test = ABTest(df, ab_params).linearization().cuped().bucketing()
ab_test.params.data_params.transforms

In [ ]:
ab_test.test_welch()

In [ ]:
ab_test.report()

In [ ]:
ab_test.plot()

# Кейс №7. Выделение групп.

In [ ]:
df = pd.read_csv('./data/ab_data.csv', nrows=5000)

In [ ]:
df["moda_city"] = np.random.randint(1, 5, df.shape[0])
df["moda_city"] = df["moda_city"].astype(str)
df["country"] = np.random.randint(1, 3, df.shape[0])
df["id"] = df.index

In [ ]:
split_builder_params = SplitBuilderParams(
    map_group_names_to_sizes={
        'control': None,
        'target': None
    },
    main_strata_col = "moda_city",
    split_metric_col = "height_now",
    metric_type = "continuous",
    id_col = "id",
    cols = ["height_prev"],
    cat_cols=["country"],
    alpha=0.05,
    n_bins = 6,
    min_cluster_size = 100
)

In [ ]:
split_builder = SplitBuilder(df, split_builder_params)

In [ ]:
split = split_builder.collect()

In [ ]:
split.head()

# Кейс №8. Препилот для непрерывной метрики.

In [ ]:
df = pd.read_csv('./data/ab_data.csv')
df = df.sample(n=20_000, random_state=1)

In [ ]:
data_params = DataParams(
    id_col='id', 
    group_col='groups', 
    control_name='A',
    treatment_name='B', 
    target='height_now', 
    is_grouped=True
)

hypothesis_params = HypothesisParams(
    alpha=0.05, 
    beta=0.2, 
    alternative='two-sided', 
    metric_type='continuous',
    metric=np.mean, 
    n_boot_samples=100, 
)

ab_params = ABTestParams(data_params, hypothesis_params)

In [ ]:
experiment_params = MdeParams(
    metrics_names=['height_now'],
    injects=[1.0001,1.0002],
    min_group_size=5000, 
    max_group_size=10000, 
    step=2500,
    variance_reduction=None,
    use_buckets=False,
    stat_test=ABTest.test_boot_confint,
    iterations_number=10,
    max_beta_score=0.9,
    min_beta_score=0.02,
)

In [ ]:
prepilot = MdeResearchBuilder(df, ab_params,
                                  experiment_params,
                                  split_builder_params)

In [ ]:
beta, alpha = prepilot.collect()

In [ ]:
beta

In [ ]:
alpha

# Кейс №9. Препилот для непрерывной метрики со снижением дисперсии.

In [ ]:
df = pd.read_csv('./data/ab_data.csv')
df = df.sample(n=20_000, random_state=1)

In [ ]:
data_params = DataParams(
    id_col='id', 
    group_col='groups', 
    control_name='A',
    treatment_name='B', 
    target='height_now', 
    covariate='height_prev',
    is_grouped=True
)

hypothesis_params = HypothesisParams(
    alpha=0.05, 
    beta=0.2, 
    alternative='two-sided',
    metric_type='continuous', 
    metric_name='mean', 
    metric=np.mean, 
    n_boot_samples=200, 
    n_buckets=50
)

ab_params = ABTestParams(data_params, hypothesis_params)

In [ ]:
prepilot_params = MdeParams(
    metrics_names=['height_now'],
    injects=[1.00046,1.00048,1.0005,1.001],
    min_group_size=5000, 
    max_group_size=10000, 
    step=1000,
    variance_reduction = ABTest.cuped,
    use_buckets = False,
    stat_test = ABTest.test_welch,
    iterations_number = 10,
    max_beta_score=0.9,
    min_beta_score=0.02,
)

In [ ]:
split_builder_params = SplitBuilderParams(
    map_group_names_to_sizes={
        'control': None,
        'target': None
    },
    main_strata_col = "moda_city",
    split_metric_col = "height_now",
    metric_type = "continuous",
    id_col = "id",
    cols = ["height_prev"],
    cat_cols=["country"],
    alpha=0.05,
    n_bins = 6,
    min_cluster_size = 100
)

In [ ]:
prepilot = MdeResearchBuilder(df, ab_params,
                                     prepilot_params,
                                     split_builder_params)

In [ ]:
beta, alpha = prepilot.collect()

In [ ]:
beta

In [ ]:
alpha

# Кейс №10. Препилот для бинарной метрики.

In [ ]:
df = pd.read_csv('./data/ab_data.csv')
df = df.sample(n=10_000, random_state=1)
df.head()

In [ ]:
data_params = DataParams(
    id_col='id', 
    group_col='groups',
    target_flg='conversion', 
    is_grouped=True
)

hypothesis_params = HypothesisParams(
    alpha=0.05, 
    beta=0.2, 
    alternative='two-sided',
    metric_type='binary', 
    metric_name='mean', 
    metric=np.mean, 
    n_boot_samples=200, 
    n_buckets=50
)

ab_params = ABTestParams(data_params, hypothesis_params)

In [ ]:
split_builder_params = SplitBuilderParams(
    map_group_names_to_sizes={
        'control': None,
        'target': None
    },
    main_strata_col = "moda_city",
    split_metric_col = "height_now",
    metric_type = "binary",
    id_col = "id",
    cols = ["height_prev"],
    cat_cols=["country"],
    alpha=0.05,
    n_bins = 6,
    min_cluster_size = 500
)

### Препилот без бакетирования

In [ ]:
prepilot_params = MdeParams(
    metrics_names=['conversion'],
    injects=[1.001,1.005,1.01],
    min_group_size=2000, 
    max_group_size=5000, 
    step=1000,
    variance_reduction = None,
    use_buckets = False,
    stat_test = ABTest.test_z_proportions,
    iterations_number = 5,
    max_beta_score=0.9,
    min_beta_score=0.01,
)

prepilot = MdeResearchBuilder(df, ab_params,
                                  prepilot_params,
                                  split_builder_params)

In [ ]:
%%time 
beta, alpha = prepilot.collect()

In [ ]:
beta

In [ ]:
alpha

# Кейс №11. Линеаризация

In [ ]:
df = pd.read_csv('./data/ab_data.csv')
df.head()

In [ ]:
data_params = DataParams(
    id_col='id', 
    group_col='groups', 
    control_name='A',
    treatment_name='B',
    numerator='numerator', 
    denominator='denominator', 
    covariate='height_prev',
    is_grouped=False
)

hypothesis_params = HypothesisParams(
    alpha=0.05, 
    beta=0.2, 
    alternative='two-sided', 
    metric_type='ratio', 
    metric_name='mean', 
    metric=np.mean, 
    n_boot_samples=200, 
    n_buckets=50
)

ab_params = ABTestParams(data_params, hypothesis_params)

In [ ]:
ab_test = ABTest(df, ab_params)
ab_test_1 = ab_test.linearization()
ab_test_2 = ab_test_1.cuped()

In [ ]:
df_linear = pd.DataFrame(data={
    'id': list(range(0, len(ab_test_1._ABTest__dataset.target_linearized), 1)),
    'ratio_linear': ab_test_1._ABTest__dataset.target_linearized
})

In [ ]:
df_grouped = df.groupby(by=['id', 'groups'], 
                        as_index=False).agg({
                            'numerator': 'sum',
                            'denominator': 'sum'
                        })
df_grouped['ratio_real'] = df_grouped.numerator / df_grouped.denominator
df_grouped = df_grouped.merge(df_linear, how='left', on='id')
df_grouped.head()

In [ ]:
df_grouped[['ratio_real', 'ratio_linear']].corr()

In [ ]:
plt.rcParams.update({'font.size': 16})
plt.figure(figsize=(10, 10))
sns.scatterplot(data=df_grouped,
            x='ratio_real', y='ratio_linear',
            hue='groups', s=80)
plt.legend()
plt.xlabel('ratio_real', fontsize=16)
plt.ylabel('ratio_linear', fontsize=16)
plt.show()
plt.close()

In [ ]:
ab_test_1.params.data_params.transforms

In [ ]:
ab_test.params.data_params.transforms

In [ ]:
print(ab_test_2.report())

# Кейс №11. Пайплайн как вызов цепочки методов

In [ ]:
data_params = DataParams(
    id_col='id', 
    group_col='groups', 
    target='height_now', 
    covariate='height_prev', 
    is_grouped=False
)

hypothesis_params = HypothesisParams(
    alpha=0.05, 
    beta=0.2, 
    alternative='greater', 
    metric_type='continuous', 
    metric_name='mean', 
    metric=np.mean, 
    n_boot_samples=100, 
    n_buckets=2000
)

ab_params = ABTestParams(data_params, hypothesis_params)

In [ ]:
df = pd.read_csv('./data/ab_data.csv')
df['height_now'].where(df['groups'] == 'A',
                       df['height_now'] * 1.003,
                       axis=0,
                       inplace=True)
df['id'] = np.random.randint(0, 150_000, df.shape[0])
df = df.drop_duplicates(['id'])

In [ ]:
ab_test = ABTest(df, ab_params).linearization().cuped().bucketing()

In [ ]:
ab_test.params.data_params.transforms

In [ ]:
ab_test.test_welch()

In [ ]:
ab_test.report()

In [ ]:
ab_test.plot()